In [1]:
# Librerías estándar y de terceros
import os
import sys
import gc
import math as m
import numpy as np
import scipy
import pandas as pd
from scipy import *
from scipy.signal import butter, filtfilt, hilbert
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

# Configuración de rutas para importaciones personalizadas
sys.path.append('/projects/DEIKE/cmartinb/jupyter_notebook/project_specific/turbulence')
sys.path.append('/projects/DEIKE/cmartinb/functions')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
# Funciones y clases personalizadas
from prepare import load_object, save_object, field
from defs import Case, Interface2D
from phase import extract_phase
from funciones import *  # Asegúrate de que la importación con * sea necesaria, a veces es mejor importar solo las funciones necesarias

os.chdir('/projects/DEIKE/cmartinb/')

In [2]:
# DEFINE CASE SO WE CAN AUTHOMATIZE IT 
kpHs = '0p04' # 0p16 
uoc = '0p50' # 0p50 0p75
reW = '2.5e4' #1.0e5
reA = 720
maxLevel = 10
Bo=200

if reW == '1.0e5':
    Re_water = 1 * 10**5
else:
    Re_water = 2.5 * 10**4

if uoc == '0p75':
    N = 1024
else:
    N = 512 

#Common parameters 
kp = 4
u = 0.5
lambdap = 2*m.pi/kp

L0 = 2*np.pi;


ak, c, omegap, nu_water, g , uoc_val= calculate_parameters(kpHs, uoc , u ,kp)
print('ak is', ak, 'c is' ,c, 'w_p$ is', omegap, '$\nu_w$ is' , nu_water, 'g is' ,g, 'N', N)

ak is 0.04 c is 1.0 w_p$ is 2.0 $
u_w$ is 6.283185307179586e-05 g is 1 N 512


In [3]:
#work_dir = f'/projects/DEIKE/nscapin/broadband_reorder/re{reA}_bo0{Bo}_P{kpHs}_uoc{uoc}_reW{reW}_L{maxLevel}/'
#for Bo=1000 I run the simulation
work_dir = f'/projects/DEIKE/cmartinb/cases_multiphase_broadbanded/re{reA}_bo{Bo}_kpHs{kpHs}_uoc{uoc}_reW{reW}_L{maxLevel}/'
data = np.loadtxt(work_dir+'eta/global_int.out')
#re720_bo0200_P0p16_uoc0p50_reW1.0e5_L10
istep_c =data[:, 1]
time = data[:,0] 
print(time.shape)

(77,)


In [4]:
eta_series = np.zeros((istep_c.shape[0],N,N), dtype=np.float32)
j=0

for i in istep_c:
    
    filename = work_dir + f'eta/eta_loc/eta_loc_t{int(i):09}.bin'
    etalo = np.fromfile(filename)
    size  = etalo.shape;
    tot_row = 18
    tot_row_i = int(size[0]/tot_row);
    etalo = etalo.reshape([tot_row_i, tot_row]);
    
    eta_m0  = 1.0; cirp_th = 0.20;
    new_row = 0;
    for i in range(tot_row_i):
        if ( abs(etalo[i][12]-eta_m0) < cirp_th ):
            new_row += 1;
    #
    print('iteration', j ,"Second pass of remove")
    etal = np.zeros([new_row, 18]);
    for i in range(new_row):
        if ( abs(etalo[i][12]-eta_m0) < cirp_th ):
            etal[i][:] = etalo[i][:];
    xarray = np.linspace(-L0/2., L0/2.,N,endpoint=False)+L0/2/N/2 # Centered grid for interpolation
    yarray = np.linspace(-L0/2., L0/2.,N,endpoint=False)+L0/2/N/2 # Centered grid for interpolation
    xtile, ytile = np.meshgrid(xarray,yarray)
    eta = griddata((etal[:,0].ravel(), etal[:,1].ravel()), etal[:,12].ravel(), (xtile, ytile), method='nearest')
    eta_series[j]= eta
    j+=1



iteration 0 Second pass of remove
iteration 1 Second pass of remove
iteration 2 Second pass of remove
iteration 3 Second pass of remove
iteration 4 Second pass of remove
iteration 5 Second pass of remove
iteration 6 Second pass of remove
iteration 7 Second pass of remove
iteration 8 Second pass of remove
iteration 9 Second pass of remove
iteration 10 Second pass of remove
iteration 11 Second pass of remove
iteration 12 Second pass of remove
iteration 13 Second pass of remove
iteration 14 Second pass of remove
iteration 15 Second pass of remove
iteration 16 Second pass of remove
iteration 17 Second pass of remove
iteration 18 Second pass of remove
iteration 19 Second pass of remove
iteration 20 Second pass of remove
iteration 21 Second pass of remove
iteration 22 Second pass of remove
iteration 23 Second pass of remove
iteration 24 Second pass of remove
iteration 25 Second pass of remove
iteration 26 Second pass of remove
iteration 27 Second pass of remove
iteration 28 Second pass of re

In [5]:
np.save(f'/projects/DEIKE/cmartinb/eta/eta_series_re{reA}_bo0{Bo}_P{kpHs}_uoc{uoc}_reW{reW}_L{maxLevel}.npy', eta_series)